In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
from __future__ import print_function

import keras
from keras.models import Sequential
from keras.layers import Conv1D, Dense, Dropout, Flatten

from loader import load_20news
from custom_layer import MonteCarloLRF

import numpy as np
import sklearn as sk

Using TensorFlow backend.


In [3]:
(input_shape, nb_classes), (X_train, X_test, Y_train, Y_test), graph_data = \
    load_20news(data_home='data', top_words=1000, sparse=False, remove_short_documents=True, verbose=False)

In [4]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2])
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2])
batch_size = 64
num_classes = Y_train.shape[1]

In [5]:
print(X_train.shape)
print(X_test.shape)
print(batch_size, num_classes)

(9922, 1000, 1)
(6695, 1000, 1)
64 20


In [6]:
METRIC = 'cosine'#'euclidean'
distances = sk.metrics.pairwise.pairwise_distances(graph_data, metric=METRIC, n_jobs=-2)

# enforce exact zero
for k in xrange(distances.shape[0]):
  distances[k,k] = 0.

In [7]:
# max normalize
#distances /= distances.max()
distances /= distances.max(axis=1).reshape((distances.shape[0], 1))

# use tricube kernel (becaause of flatness around 0)
probabilities = (1. - np.abs(distances) ** 3) ** 3

# remove auto connections (which are taken anyway in LRF)
for k in xrange(probabilities.shape[0]):
  probabilities[k,k] = 0.

# normalize proba
probabilities /= np.sum(probabilities, axis=1).reshape((probabilities.shape[0], 1))

In [ ]:
model = Sequential()
model.add(MonteCarloLRF(probabilities, LRF_size=16, filters=64, activation='relu',
                       input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
monte_carlo_lrf_1 (MonteCarl (None, 1000, 64)          1088      
_________________________________________________________________
dropout_1 (Dropout)          (None, 1000, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 64000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 20)                1280020   
Total params: 1,281,108
Trainable params: 1,281,108
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(X_train, Y_train,
                    batch_size=batch_size,
                    epochs=20,
                    verbose=1,
                    validation_data=(X_test, Y_test))
score = model.evaluate(X_test, Y_test, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 9922 samples, validate on 6695 samples
Epoch 1/20
9922/9922 [==============================] - 16s - loss: 2.1623 - acc: 0.4728 - val_loss: 1.7795 - val_acc: 0.5231
Epoch 2/20
9922/9922 [==============================] - 15s - loss: 1.2539 - acc: 0.6742 - val_loss: 1.5492 - val_acc: 0.5713
Epoch 3/20
9922/9922 [==============================] - 15s - loss: 0.9703 - acc: 0.7295 - val_loss: 1.5562 - val_acc: 0.5630
Epoch 4/20
9922/9922 [==============================] - 15s - loss: 0.8200 - acc: 0.7627 - val_loss: 1.6100 - val_acc: 0.5658
Epoch 5/20
9922/9922 [==============================] - 15s - loss: 0.7130 - acc: 0.7906 - val_loss: 1.7034 - val_acc: 0.5516
Epoch 6/20
9922/9922 [==============================] - 15s - loss: 0.6332 - acc: 0.8127 - val_loss: 1.8006 - val_acc: 0.5476
Epoch 7/20
9922/9922 [==============================] - 15s - loss: 0.5724 - acc: 0.8298 - val_loss: 1.8846 - val_acc: 0.5459
Epoch 8/20
9922/9922 [==============================] - 15s - loss: 0.